In [1]:
import sklearn as skl
import pandas as pd
import xgboost as xgb
import numpy as np
entrenamiento = pd.read_csv('Train_TP2_Datos_2020-2C.csv')
test = pd.read_csv('Test_TP2_Datos_2020-2C.csv')

In [2]:
##ENTRENAMIENTO

entrenamiento = entrenamiento[( entrenamiento['Stage'] == 'Closed Won') | ( entrenamiento['Stage'] == 'Closed Lost')]


#QUITO COLUMNAS INÚTILES
entrenamiento = entrenamiento.drop(columns=['ASP_(converted)_Currency','Quote_Type','Brand','Product_Type','Size','Product_Category_B','Price','Currency','Last_Activity','Actual_Delivery_Date','Prod_Category_A'])

#QUITO COLUMNAS CON DEMASIADOS VALORES
entrenamiento = entrenamiento.drop(columns=['Opportunity_Name','Product_Family','Sales_Contract_No','Product_Name','Delivery_Quarter','Total_Amount_Currency','Total_Taxable_Amount_Currency'])

#TRANSFORMO A DIFERENCIA DE DIAS LAS FECHAS
def formateo(x):
    if(x == 'NaT'):
        return x
    cadena = x.split('/')
    return '{dia:02}/{mes:02}/{ano}'.format(dia=int(cadena[0]), mes=int(cadena[1]),ano=int(cadena[2]))
columnas_fecha = ['Last_Modified_Date','Account_Created_Date','Opportunity_Created_Date','Quote_Expiry_Date','Planned_Delivery_Start_Date','Planned_Delivery_End_Date']
for columna in columnas_fecha:
    entrenamiento[columna] = entrenamiento[columna].apply(formateo)
    entrenamiento[columna] = pd.to_datetime(entrenamiento[columna], format='%m/%d/%Y')

entrenamiento = entrenamiento.dropna()

def tiempo_en_dias(x):
    return (x - pd.to_datetime('01/01/2000', format='%m/%d/%Y')).days
for columna in columnas_fecha:
    entrenamiento[columna] = entrenamiento[columna].apply(tiempo_en_dias)
    entrenamiento[columna] = entrenamiento[columna].astype('int32')

#TRATAMIENTO ESPECIAL A MONTH
def formateo_mes(x):
    x = x.replace(" - ","/")
    cadena = x.split('/')
    return '{ano}/{mes:02}'.format(ano=int(cadena[0]), mes=int(cadena[1]))
entrenamiento['Month'] = entrenamiento['Month'].apply(formateo_mes)
entrenamiento['Month'] = pd.to_datetime(entrenamiento['Month'], format='%Y/%m')
entrenamiento['Month'] = entrenamiento['Month'].apply(tiempo_en_dias)
entrenamiento['Month'] = entrenamiento['Month'].astype('int32')



In [3]:
##TEST

#QUITO COLUMNAS INÚTILES
test = test.drop(columns=['ASP_(converted)_Currency','Quote_Type','Brand','Product_Type','Size','Product_Category_B','Price','Currency','Last_Activity','Actual_Delivery_Date','Prod_Category_A'])

#QUITO COLUMNAS CON DEMASIADOS VALORES
test = test.drop(columns=['Opportunity_Name','Product_Family','Sales_Contract_No','Product_Name','Delivery_Quarter','Total_Amount_Currency','Total_Taxable_Amount_Currency'])

#TRANSFORMO A DIFERENCIA DE DIAS LAS FECHAS
def formateo(x):
    if(x == 'NaT'):
        return x
    cadena = x.split('/')
    return '{dia:02}/{mes:02}/{ano}'.format(dia=int(cadena[0]), mes=int(cadena[1]),ano=int(cadena[2]))
columnas_fecha = ['Last_Modified_Date','Account_Created_Date','Opportunity_Created_Date','Quote_Expiry_Date','Planned_Delivery_Start_Date','Planned_Delivery_End_Date']
for columna in columnas_fecha:
    test[columna] = test[columna].apply(formateo)
    test[columna] = pd.to_datetime(test[columna], format='%m/%d/%Y')

test = test.dropna()

def tiempo_en_dias(x):
    return (x - pd.to_datetime('01/01/2000', format='%m/%d/%Y')).days
for columna in columnas_fecha:
    test[columna] = test[columna].apply(tiempo_en_dias)
    test[columna] = test[columna].astype('int32')

#TRATAMIENTO ESPECIAL A MONTH
def formateo_mes(x):
    x = x.replace(" - ","/")
    cadena = x.split('/')
    return '{ano}/{mes:02}'.format(ano=int(cadena[0]), mes=int(cadena[1]))
test['Month'] = test['Month'].apply(formateo_mes)
test['Month'] = pd.to_datetime(test['Month'], format='%Y/%m')
test['Month'] = test['Month'].apply(tiempo_en_dias)
test['Month'] = test['Month'].astype('int32')

In [4]:
#QUITO STAGE
objetivo = entrenamiento.pop('Stage')
entrenamiento.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10086 entries, 0 to 16946
Data columns (total 33 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   ID                                  10086 non-null  int64  
 1   Region                              10086 non-null  object 
 2   Territory                           10086 non-null  object 
 3   Pricing, Delivery_Terms_Quote_Appr  10086 non-null  int64  
 4   Pricing, Delivery_Terms_Approved    10086 non-null  int64  
 5   Bureaucratic_Code_0_Approval        10086 non-null  int64  
 6   Bureaucratic_Code_0_Approved        10086 non-null  int64  
 7   Submitted_for_Approval              10086 non-null  int64  
 8   Bureaucratic_Code                   10086 non-null  object 
 9   Account_Created_Date                10086 non-null  int32  
 10  Source                              10086 non-null  object 
 11  Billing_Country                     10086

In [5]:
def se_gano(x):
    return int(x == 'Closed Won')
objetivo = objetivo.apply(se_gano)
objetivo

0        0
1        1
2        1
4        0
5        0
        ..
16939    1
16940    0
16941    0
16945    0
16946    0
Name: Stage, Length: 10086, dtype: int64

In [6]:
columnas_object = list(entrenamiento.select_dtypes(include=['object']).columns)

for columna in columnas_object:
    print(columna + ":" + str(entrenamiento[columna].value_counts().size))

Region:5
Territory:75
Bureaucratic_Code:6
Source :14
Billing_Country:77
Account_Name:1363
Account_Owner:46
Opportunity_Owner:52
Account_Type:7
Opportunity_Type:25
Delivery_Terms:9
Last_Modified_By:51
ASP_Currency:5


In [7]:
def categoricas_a_numericas(x):
    ohe = skl.preprocessing.OrdinalEncoder()
    for columna in columnas_object:
        copia = x[[columna]].copy().dropna()
        df_temp = pd.DataFrame(ohe.fit_transform(copia)).astype('int32')
        df_temp.columns = [columna]
        x[columna] = df_temp[columna]
        print(columna)
categoricas_a_numericas(entrenamiento)
categoricas_a_numericas(test)

Region
Territory
Bureaucratic_Code
Source 
Billing_Country
Account_Name
Account_Owner
Opportunity_Owner
Account_Type
Opportunity_Type
Delivery_Terms
Last_Modified_By
ASP_Currency
Region
Territory
Bureaucratic_Code
Source 
Billing_Country
Account_Name
Account_Owner
Opportunity_Owner
Account_Type
Opportunity_Type
Delivery_Terms
Last_Modified_By
ASP_Currency


In [8]:
entrenamiento.info()
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10086 entries, 0 to 16946
Data columns (total 33 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   ID                                  10086 non-null  int64  
 1   Region                              6550 non-null   float64
 2   Territory                           6550 non-null   float64
 3   Pricing, Delivery_Terms_Quote_Appr  10086 non-null  int64  
 4   Pricing, Delivery_Terms_Approved    10086 non-null  int64  
 5   Bureaucratic_Code_0_Approval        10086 non-null  int64  
 6   Bureaucratic_Code_0_Approved        10086 non-null  int64  
 7   Submitted_for_Approval              10086 non-null  int64  
 8   Bureaucratic_Code                   6550 non-null   float64
 9   Account_Created_Date                10086 non-null  int32  
 10  Source                              6550 non-null   float64
 11  Billing_Country                     6550 

In [9]:
d_entrenamiento = xgb.DMatrix(entrenamiento.values, objetivo.values)
d_prueba = xgb.DMatrix(test.values)

In [10]:
param = {'max_depth':2, 'eta':1, 'objective':'binary:logistic' }
num_round = 2
bst = xgb.train(param, d_entrenamiento, num_round)

[18:01:33] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [11]:
preds = bst.predict(d_prueba)

In [18]:
resultados = pd.Series([int(x >= 0.5) for x in list(preds)])
resultados

0       1
1       1
2       1
3       0
4       1
       ..
1255    1
1256    1
1257    1
1258    1
1259    0
Length: 1260, dtype: int64

In [19]:
resultados.value_counts()

1    840
0    420
dtype: int64